In [1]:
import pandas as pd
import numpy as np

# Download Est 2022_04 Data

In [2]:
est_04_data = pd.read_excel(r"C:\Users\cra\OneDrive - San Diego Association of Governments\Working_Drive\concept_comparison\region_est_2022_04_age_sex_ethnicity_ind_QA.xlsx")
est_04_data

,region,yr_id,age group,sex,Hispanic,"Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Asian","Non-Hispanic, Black","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, Other","Non-Hispanic, Two or More Races","Non-Hispanic, White"
0,San Diego,2020,10 to 14,Female,48666,389,10073,4244,261,422,9874,33129
1,San Diego,2020,10 to 14,Male,49591,343,10736,4909,414,443,10703,34738
2,San Diego,2020,15 to 17,Female,27335,206,6435,2627,185,247,5318,19395
3,San Diego,2020,15 to 17,Male,28878,211,6900,2699,185,270,5371,21200
4,San Diego,2020,18 and 19,Female,18749,147,5075,2039,162,186,3641,14675
...,...,...,...,...,...,...,...,...,...,...,...,...
115,San Diego,2022,80 to 84,Male,4207,132,3144,895,95,212,507,16651
116,San Diego,2022,85 and Older,Female,7091,188,5507,1032,142,327,664,25660
117,San Diego,2022,85 and Older,Male,3644,82,2802,597,122,210,286,16500
118,San Diego,2022,Under 5,Female,37588,310,10190,4449,377,396,6327,31059


# Donwlaod and Prep Concept Data

In [5]:
concept_data = pd.read_excel(r"J:\DataScience\Estimates&Forecasts\SR15 Forecast\SR15_CCM\CCM_OutputData\DataToEconTeam\DataToEconTeam07072023\Household_ characteristics_2020_2060_forQA_07072023.xlsx", sheet_name="data", usecols=['race', 'sex', 'age', 'year', 'control_pop'])
concept_data = concept_data[concept_data['year'].isin([2020, 2021, 2022])]

In [7]:

age_bins = [-1, 4, 9, 14, 17, 19, 24, 29, 34, 39, 44, 49, 54, 59, 61, 64, 69, 74, 79, 84, float('inf')]
age_labels = ['Under 5', '5 to 9', '10 to 14', '15 to 17', '18 and 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54', '55 to 59', '60 and 61', '62 to 64', '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85 and Older']

concept_data['age group'] = pd.cut(concept_data['age'], bins=age_bins, labels=age_labels)

In [8]:
concept_data['sex'] = concept_data['sex'].replace({'F': 'Female', 'M': 'Male'})

In [9]:
concept_data['race_long_name'] = concept_data['race'].replace({
            'AIAN_NH': 'Non-Hispanic, American Indian or Alaska Native', 
            'Asian_NH': 'Non-Hispanic, Asian', 
            'Black_NH': 'Non-Hispanic, Black', 
            'Hispanic': 'Hispanic', 
            'Multi-Race NH': 'Non-Hispanic, Two or More Races', 
            'NHPI_NH': 'Non-Hispanic, Hawaiian or Pacific Islander', 
            'White_NH': 'Non-Hispanic, White'})

In [10]:
concept_data_cleaned = concept_data[['sex', 'year', 'control_pop', 'age group','race_long_name']]
concept_data_cleaned['region'] = 'San Diego'
concept_data_cleaned.columns = ['sex', 'yr_id', 'population', 'age group', 'race', 'region']

C:\Users\cra\AppData\Local\Temp\ipykernel_16716\2975591468.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concept_data_cleaned['region'] = 'San Diego'


In [11]:
concept_data_cleaned = concept_data_cleaned.groupby(['sex', 'yr_id', 'age group', 'race', 'region']).sum().reset_index()
concept_data_cleaned

,sex,yr_id,age group,race,region,population
0,Female,2020,Under 5,Hispanic,San Diego,43268
1,Female,2020,Under 5,"Non-Hispanic, American Indian or Alaska Native",San Diego,348
2,Female,2020,Under 5,"Non-Hispanic, Asian",San Diego,9688
3,Female,2020,Under 5,"Non-Hispanic, Black",San Diego,4261
4,Female,2020,Under 5,"Non-Hispanic, Hawaiian or Pacific Islander",San Diego,347
...,...,...,...,...,...,...
835,Male,2022,85 and Older,"Non-Hispanic, Asian",San Diego,2802
836,Male,2022,85 and Older,"Non-Hispanic, Black",San Diego,597
837,Male,2022,85 and Older,"Non-Hispanic, Hawaiian or Pacific Islander",San Diego,122
838,Male,2022,85 and Older,"Non-Hispanic, Two or More Races",San Diego,286


In [14]:
result = pd.pivot(concept_data_cleaned, values='population', index=['region', 'yr_id', 'age group', 'sex'], columns=['race'])
result.columns.name = ''
result = result.reset_index()
result
#result.to_excel(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-040 CCM Forecast Output (series 15)\Results\2020-2060\Version 3\concep_data_in_estimates_format.xlsx', index=False)

# Comparing Age

In [15]:
# Age in Estimates 2022_04 Data
age_data = est_04_data[['age group', 'Hispanic',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Black', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, Other', 'Non-Hispanic, Two or More Races',
       'Non-Hispanic, White']]

age_data = age_data.groupby('age group').sum().reset_index()
age_data['total'] = age_data.drop('age group', axis=1).sum(axis=1)
age_data = age_data[['age group', 'total']]
age_data

,age group,total
0,10 to 14,646302
1,15 to 17,377506
2,18 and 19,275324
3,20 to 24,749716
4,25 to 29,731705
5,30 to 34,699174
6,35 to 39,694880
7,40 to 44,631138
8,45 to 49,602203
9,5 to 9,636812


In [16]:
# Age group in concept data 
age_data_concept = result[['age group', 'Hispanic',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Black', 'Non-Hispanic, Hawaiian or Pacific Islander', 'Non-Hispanic, Two or More Races',
       'Non-Hispanic, White']]

age_data_concept = age_data_concept.groupby('age group').sum().reset_index()
age_data_concept['total'] = age_data_concept.drop('age group', axis=1).sum(axis=1)
age_data_concept = age_data_concept[['age group', 'total']]
age_data_concept

,age group,total
0,Under 5,589901
1,5 to 9,636812
2,10 to 14,646302
3,15 to 17,377506
4,18 and 19,275324
5,20 to 24,749716
6,25 to 29,731705
7,30 to 34,699174
8,35 to 39,694880
9,40 to 44,631138


In [17]:
# Join and Find The difference
age_data_merge = age_data.merge(age_data_concept, how='left', on='age group', suffixes=['_est', '_concept'])
age_data_merge['diff'] = age_data_merge['total_est'] - age_data_merge['total_concept']
#age_data_merge.to_excel('age_differences.xlsx')
age_data_merge

,age group,total_est,total_concept,diff
0,10 to 14,646302,646302,0
1,15 to 17,377506,377506,0
2,18 and 19,275324,275324,0
3,20 to 24,749716,749716,0
4,25 to 29,731705,731705,0
5,30 to 34,699174,699174,0
6,35 to 39,694880,694880,0
7,40 to 44,631138,631138,0
8,45 to 49,602203,602203,0
9,5 to 9,636812,636812,0


# Sex Comparison

In [18]:
# Sex in Estimates 2022_04 Data
sex_data_est = est_04_data[['sex', 'Hispanic',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Black', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, Other', 'Non-Hispanic, Two or More Races',
       'Non-Hispanic, White']]

sex_data_est = sex_data_est.groupby('sex').sum().reset_index()
sex_data_est['total'] = sex_data_est.drop('sex', axis=1).sum(axis=1)
sex_data_est = sex_data_est[['sex', 'total']]
sex_data_est

,sex,total
0,Female,4932047
1,Male,4975041


In [19]:
# Sex group in concept data 
sex_data_concept = result[['sex', 'Hispanic',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Black', 'Non-Hispanic, Hawaiian or Pacific Islander', 'Non-Hispanic, Two or More Races',
       'Non-Hispanic, White']]

sex_data_concept = sex_data_concept.groupby('sex').sum().reset_index()
sex_data_concept['total'] = sex_data_concept.drop('sex', axis=1).sum(axis=1)
sex_data_concept = sex_data_concept[['sex', 'total']]
sex_data_concept

,sex,total
0,Female,4932047
1,Male,4975041


In [20]:
# Join and Find The difference
sex_data_merge = sex_data_est.merge(sex_data_concept, how='left', on='sex', suffixes=['_est', '_concept'])
sex_data_merge['diff'] = sex_data_merge['total_est'] - sex_data_merge['total_concept']
#age_data_merge.to_excel('age_differences.xlsx')
sex_data_merge

,sex,total_est,total_concept,diff
0,Female,4932047,4932047,0
1,Male,4975041,4975041,0


# Ethnicity Comparison

In [21]:
est_eth = pd.DataFrame(est_04_data.sum(axis=0))

est_eth.loc[['Hispanic',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Black', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, Other', 'Non-Hispanic, Two or More Races',
       'Non-Hispanic, White']]

,0
Hispanic,3371331
"Non-Hispanic, American Indian or Alaska Native",38413
"Non-Hispanic, Asian",1228345
"Non-Hispanic, Black",433910
"Non-Hispanic, Hawaiian or Pacific Islander",39569
"Non-Hispanic, Other",54063
"Non-Hispanic, Two or More Races",499235
"Non-Hispanic, White",4242222


In [50]:
4242222+54063

4296285

: 

In [22]:
concept_output = pd.DataFrame(result.sum(axis=0))

concept_output.loc[['Hispanic',
       'Non-Hispanic, American Indian or Alaska Native', 'Non-Hispanic, Asian',
       'Non-Hispanic, Black', 'Non-Hispanic, Hawaiian or Pacific Islander',
       'Non-Hispanic, Two or More Races', 'Non-Hispanic, White']]

C:\Users\cra\AppData\Local\Temp\ipykernel_16716\729850177.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  concept_output = pd.DataFrame(result.sum(axis=0))


,0
,
Hispanic,3371331
"Non-Hispanic, American Indian or Alaska Native",38413
"Non-Hispanic, Asian",1228345
"Non-Hispanic, Black",433910
"Non-Hispanic, Hawaiian or Pacific Islander",39569
"Non-Hispanic, Two or More Races",499235
"Non-Hispanic, White",4296285
